In [ ]:
%serialconnect

In [ ]:
import network
import ubinascii
import espnow
import time

In [3]:
def setupESPNow():
    sta = network.WLAN(network.STA_IF)
    sta.active(True)
    sta.disconnect()
    
    try:
        e = espnow.ESPNow() #Enable ESPNow
        e.active(True)
        print("ESPNow Setup Successfully")
        return(e)
    except:
        print("Failure to setup ESPNow")
        return(0)

In [37]:
def getMyMacAddr():
    try: 
        wlan_sta = network.WLAN(network.STA_IF)
        wlan_sta.active(True)
        wlan_mac = wlan_sta.config('mac')
        (wlan_mac)
        print(f"Success : {ubinascii.hexlify(wlan_mac).decode()}")
        return(type(wlan_mac))
    except:
        print("Could not get mac address for board....")
    
myMacAddr = getMyMacAddr()
print(f"My Mac Addr : {myMacAddr}")

Success : 348518abdc90
My Mac Addr : <class 'bytes'>


In [46]:
def setupEspPeer(peer, e):
    print(peer)
    try:
        e.add_peer(peer) 
        print(f"Peer added : {peer}")
    except:
        print(f"Could not add peer  : {peer}")
    return(e)          

In [48]:
def sendEspDataToPeers(peer, data, e):
    try:
        e.send(peer, data, True)
        return(1)
    except:
        print(f"Could not send data to peer : {peer}")
        return(0)


In [49]:
#Main 
import json
ESPStatus = "Master"
peerMAC = b'\xe8\x68\xe7\x4e\xbb\x19'
e = setupESPNow()
e = setupEspPeer(peerMAC, e)
print(e)

data = "Test Data for ESPNow"
sendStatus = sendEspDataToPeers(peerMAC, data, e)


ESPNow Setup Successfully
b'\xe8h\xe7N\xbb\x19'
Could not add peer  : b'\xe8h\xe7N\xbb\x19'
<ESPNow object at 3fcb1820>


In [8]:
help("modules")

__main__          espnow            uasyncio/lock     upysh
_boot             flashbdev         uasyncio/stream   urandom
_espnow           framebuf          ubinascii         ure
_onewire          gc                ubluetooth        urequests
_secrets          inisetup          ucollections      uselect
_thread           math              ucryptolib        usocket
_uasyncio         micropython       uctypes           ussl
_webrepl          mip/__init__      uerrno            ustruct
aioespnow         neopixel          uhashlib          usys
apa106            network           uheapq            utime
btree             ntptime           uio               utimeq
builtins          onewire           ujson             uwebsocket
cmath             uarray            umachine          uzlib
dht               uasyncio/__init__ umqtt/robust      webrepl
ds18x20           uasyncio/core     umqtt/simple      webrepl_setup
esp               uasyncio/event    uos
esp32             uasyncio/funcs    

In [44]:
peerMAC = b'\xe8\x68\xe7\x4e\xbb\x19'

print(peerMAC.hex())

asciiMAC = peerMAC.hex()

print(bytes.fromhex(asciiMAC))

e868e74ebb19
b'\xe8h\xe7N\xbb\x19'


In [45]:
help(bytes)

object <class 'bytes'> is of type type
  hex -- <function>
  fromhex -- <classmethod>
  decode -- <function>
  find -- <function>
  rfind -- <function>
  index -- <function>
  rindex -- <function>
  join -- <function>
  split -- <function>
  splitlines -- <function>
  rsplit -- <function>
  startswith -- <function>
  endswith -- <function>
  strip -- <function>
  lstrip -- <function>
  rstrip -- <function>
  format -- <function>
  replace -- <function>
  count -- <function>
  partition -- <function>
  rpartition -- <function>
  center -- <function>
  lower -- <function>
  upper -- <function>
  isspace -- <function>
  isalpha -- <function>
  isdigit -- <function>
  isupper -- <function>
  islower -- <function>
